In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re

## Merged CREs

In [9]:
merged_cres = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Gene_Mapping/refs/unique_cres.csv')

In [10]:
merged_cres

,Unnamed: 0,chr,start,end
0,1,chr1,797406.0,798178.0
1,2,chr1,816037.0,818244.0
2,3,chr1,819296.0,820336.0
3,4,chr1,826435.0,828307.0
4,5,chr1,842686.0,843233.0
...,...,...,...,...
200044,200045,chrY,21421806.0,21422749.0
200045,200046,chrY,21440434.0,21441017.0
200046,200047,chrY,21481413.0,21482088.0
200047,200048,chrY,26408715.0,26409392.0


In [11]:
merged_cres = merged_cres.drop(columns=['Unnamed: 0'])
merged_cres['chr'] = merged_cres['chr'].str.replace('chr', '')

In [13]:
merged_cres['start'] = merged_cres['start'].astype(int)
merged_cres['end'] = merged_cres['end'].astype(int)

In [15]:
merged_cres.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/merged_cres.csv', index=False)

## Merged CRE TFs table

Parse through cre transcription factor file to get 6 columns
- tf
- merged cre (chr, start, end)
- condition
- cell

In [16]:
merged_cres_tfs = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Gene_Mapping/refs/TFBSlog2FCsum_mergedCRE_deAD_sigTF_top0.05dis_neat.txt', sep='\t')

In [17]:
merged_cres_tfs.columns = [
    merged_cres_tfs.columns[0],  # Keep the first column's name
    merged_cres_tfs.columns[1],  # Keep the second column's name
    *[re.sub(r'_(.*?)_', '-', col) for col in merged_cres_tfs.columns[2:]]  # Apply the regex to the rest
]

In [80]:
merged_cres_tfs.head()

,chr,start,end,AhrArnt-H1-IFN,Arntl-H1-IFN,BHLHE40-H1-IFN,BHLHE41-H1-IFN,E2F8-H1-IFN,EGR1-H1-IFN,EGR3-H1-IFN,...,ZNF35-iPSC-coculture,ZNF454-iPSC-coculture,ZNF460-iPSC-coculture,ZNF530-iPSC-coculture,ZNF558-iPSC-coculture,ZNF610-iPSC-coculture,ZNF740-iPSC-coculture,ZNF8-iPSC-coculture,ZNF93-iPSC-coculture,Zfp961-iPSC-coculture
0,chr1,797406,798178,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chr1,816037,818244,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,chr1,819296,820336,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chr1,826435,828307,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,chr1,842686,843233,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
def parse_merged_cres_faster(df):
    # Extract columns representing chromosome, start, and end
    chr_col = df['chr'].values
    start_col = df['start'].values
    end_col = df['end'].values
    
    # Get all column names except 'chr', 'start', 'end'
    feature_cols = [col for col in df.columns if col not in ['chr', 'start', 'end']]
    
    # Prepare lists to collect data
    keys = []
    values = []
    
    # Process each feature column
    for col_name in feature_cols:
        # Find indices where value is 1
        indices = np.where(df[col_name].values == 1)[0]
        
        if len(indices) > 0:
            names = col_name.split('-')
            
            # Prepare the cell and condition values based on column name pattern
            if len(names) == 2:
                condition = names[1]
                if condition in ['SORL1A528T', 'SORL1KO', 'TREM2R47H', 'TREM2KO', 'CD33', 'INPP5D']:
                    cell = 'ESC'
                elif condition in ['APOE4vs2', 'APOE4vs3', 'APOEKOvs2', 'APOEKOvs3', 'HIVactivated', 'HIVlatent']:
                    cell = 'iPSC'
                else:
                    cell = 'Unknown'
                tf, cell_type, cond = names[0], cell, condition
            elif len(names) == 3:
                cell = names[1]
                condition = names[2]
                if condition in ['ACTvsLAT', 'CRISPR','ACSL1']:
                    cell = 'iPSC'
                    condition = f'{names[1]}-{names[2]}'
                tf, cell_type, cond = names[0], cell, condition
            elif len(names) == 4:
                tf, cell_type, cond = f'{names[0]}-{names[1]}', names[2], names[3]
            else:
                continue  # Skip if pattern doesn't match
            
            # For each index where value is 1, create a record
            for idx in indices:
                keys.append(f'{idx}_{col_name}')
                values.append([chr_col[idx], start_col[idx], end_col[idx], tf, cell_type, cond])
    
    # Create dataframe from collected data
    result_df = pd.DataFrame(values, index=keys, 
                            columns=[0, 1, 2, 3, 4, 5])
    
    return result_df

In [19]:
merged_cres_tfs_trial = merged_cres_tfs.head(2)

In [33]:
parse_merged_cres_faster(merged_cres_tfs_trial)

,0,1,2,3,4,5
1_Bach1Mafk-WTC11-IFN,chr1,816037,818244,Bach1Mafk,WTC11,IFN
1_Erg-WTC11-IFN,chr1,816037,818244,Erg,WTC11,IFN
1_GABPA-WTC11-IFN,chr1,816037,818244,GABPA,WTC11,IFN
1_IKZF2-WTC11-IFN,chr1,816037,818244,IKZF2,WTC11,IFN
1_IRF2-WTC11-IFN,chr1,816037,818244,IRF2,WTC11,IFN
...,...,...,...,...,...,...
1_Nrf1-iPSC-coculture,chr1,816037,818244,Nrf1,iPSC,coculture
1_SP3-iPSC-coculture,chr1,816037,818244,SP3,iPSC,coculture
1_SPIB-iPSC-coculture,chr1,816037,818244,SPIB,iPSC,coculture
1_SPIC-iPSC-coculture,chr1,816037,818244,SPIC,iPSC,coculture


In [34]:
mergedcre_tf = parse_merged_cres_faster(merged_cres_tfs)

In [35]:
mergedcre_tf.columns = ['chr', 'start', 'end', 'transcription_factor', 'cell_type', 'condition']

In [36]:
mergedcre_tf

,chr,start,end,transcription_factor,cell_type,condition
3_AhrArnt-H1-IFN,chr1,826435,828307,AhrArnt,H1,IFN
94_AhrArnt-H1-IFN,chr1,1397971,1400475,AhrArnt,H1,IFN
124_AhrArnt-H1-IFN,chr1,1614037,1616691,AhrArnt,H1,IFN
142_AhrArnt-H1-IFN,chr1,1745525,1747386,AhrArnt,H1,IFN
218_AhrArnt-H1-IFN,chr1,2226872,2228282,AhrArnt,H1,IFN
...,...,...,...,...,...,...
199972_Zfp961-iPSC-coculture,chrY,13751202,13752042,Zfp961,iPSC,coculture
199988_Zfp961-iPSC-coculture,chrY,14523314,14525373,Zfp961,iPSC,coculture
199992_Zfp961-iPSC-coculture,chrY,14840305,14840848,Zfp961,iPSC,coculture
200025_Zfp961-iPSC-coculture,chrY,19075637,19076488,Zfp961,iPSC,coculture


In [37]:
mergedcre_tf['cell_type'] = mergedcre_tf['cell_type'].replace('H1', 'ESC')
mergedcre_tf['cell_type'] = mergedcre_tf['cell_type'].replace('WTC11', 'iPSC')

In [38]:
mergedcre_tf['chr'] = mergedcre_tf['chr'].str.replace('chr', '')

In [39]:
mergedcre_tf

,chr,start,end,transcription_factor,cell_type,condition
3_AhrArnt-H1-IFN,1,826435,828307,AhrArnt,ESC,IFN
94_AhrArnt-H1-IFN,1,1397971,1400475,AhrArnt,ESC,IFN
124_AhrArnt-H1-IFN,1,1614037,1616691,AhrArnt,ESC,IFN
142_AhrArnt-H1-IFN,1,1745525,1747386,AhrArnt,ESC,IFN
218_AhrArnt-H1-IFN,1,2226872,2228282,AhrArnt,ESC,IFN
...,...,...,...,...,...,...
199972_Zfp961-iPSC-coculture,Y,13751202,13752042,Zfp961,iPSC,coculture
199988_Zfp961-iPSC-coculture,Y,14523314,14525373,Zfp961,iPSC,coculture
199992_Zfp961-iPSC-coculture,Y,14840305,14840848,Zfp961,iPSC,coculture
200025_Zfp961-iPSC-coculture,Y,19075637,19076488,Zfp961,iPSC,coculture


### Filter wanted conditions

In [40]:
mergedcre_tf['condition'].unique()

array(['IFN', 'SORL1A528T', 'SORL1KO', 'TREM2R47H', 'TREM2KO', 'CD33',
       'INPP5D', 'APOE3-ACSL1', 'APOE4vs2', 'APOE4vs3', 'APOEKOvs2',
       'APOEKOvs3', 'CLU-CRISPR', 'HIVactivated', 'HIV-ACTvsLAT',
       'HIVlatent', 'xenot8w', 'xenot12d', 'xenot7d', 'coculture'],
      dtype=object)

In [41]:
first_rows = mergedcre_tf.groupby('condition', as_index=False).first()
print(first_rows)

       condition chr     start       end transcription_factor cell_type
0    APOE3-ACSL1   1    816037    818244               Bcl11B      iPSC
1       APOE4vs2   1    826435    828307              AhrArnt      iPSC
2       APOE4vs3   1   2289848   2291601                 Alx1      iPSC
3      APOEKOvs2   1    826435    828307              AhrArnt      iPSC
4      APOEKOvs3   1    826435    828307              AhrArnt      iPSC
5           CD33   1   1157014   1158958               Arid3a       ESC
6     CLU-CRISPR   1  17128174  17129001                 Alx1      iPSC
7   HIV-ACTvsLAT   1    869480    870292                ASCL1      iPSC
8   HIVactivated   1    937942    938810                 Atf3      iPSC
9      HIVlatent   1    869480    870292                ASCL1      iPSC
10           IFN   1    826435    828307              AhrArnt       ESC
11        INPP5D   1   8702524   8704123               Arid3b       ESC
12    SORL1A528T   1    826435    828307                ASCL1   

In [46]:
merged_df = mergedcre_tf[mergedcre_tf['condition'].isin(['IFN', 'TREM2R47H', 'TREM2KO', 'xenot7d', 'coculture'])]

In [53]:
merged_df['transcription_factor'] = merged_df['transcription_factor'].str.upper()
merged_df

/var/folders/y_/tfywlws16k71l2k62r0j683m0000gn/T/ipykernel_7367/3909949217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['transcription_factor'] = merged_df['transcription_factor'].str.upper()


,chr,start,end,transcription_factor,cell_type,condition
3_AhrArnt-H1-IFN,1,826435,828307,AHRARNT,ESC,IFN
94_AhrArnt-H1-IFN,1,1397971,1400475,AHRARNT,ESC,IFN
124_AhrArnt-H1-IFN,1,1614037,1616691,AHRARNT,ESC,IFN
142_AhrArnt-H1-IFN,1,1745525,1747386,AHRARNT,ESC,IFN
218_AhrArnt-H1-IFN,1,2226872,2228282,AHRARNT,ESC,IFN
...,...,...,...,...,...,...
199972_Zfp961-iPSC-coculture,Y,13751202,13752042,ZFP961,iPSC,coculture
199988_Zfp961-iPSC-coculture,Y,14523314,14525373,ZFP961,iPSC,coculture
199992_Zfp961-iPSC-coculture,Y,14840305,14840848,ZFP961,iPSC,coculture
200025_Zfp961-iPSC-coculture,Y,19075637,19076488,ZFP961,iPSC,coculture


In [54]:
merged_df.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/merged_cre_tf.csv', index=False)

## Transcription Factors

In [52]:
tfs = pd.DataFrame(mergedcre_tf['transcription_factor'].unique(), columns=['transcription_factor'])
tfs['transcription_factor'] = tfs['transcription_factor'].str.upper()
tfs

,transcription_factor
0,AHRARNT
1,ARNTL
2,BHLHE40
3,BHLHE41
4,E2F8
...,...
520,NKX6-1
521,MIX-A
522,CDX1
523,NFATC4


In [55]:
tfs.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/tfs.csv', index=False)

## Conditions

In [ ]:
conditions = pd.DataFrame(mergedcre_tf['condition'].unique(), columns=['condition'])
conditions

,condition
0,IFN
1,SORL1A528T
2,SORL1KO
3,TREM2R47H
4,TREM2KO
5,CD33
6,INPP5D
7,APOE3-ACSL1
8,APOE4vs2
9,APOE4vs3


In [60]:
#conditions['disease'] = 'AD'
conditions.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/conditions.csv', index=False)

## Cell

In [66]:
cell = pd.DataFrame(mergedcre_tf['cell_type'].unique(), columns=['cell_type'])
cell

,cell_type
0,ESC
1,iPSC
2,1


In [67]:
cell.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/cell_type.csv', index=False)

## Genes 

In [48]:
genes_inf = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Genes/ifnb_wtc11_genes.csv', index_col=0)
genes_inf

,hgnc,ensembl_gene_id,entrez,chromosome,start_position,end_position,strand
1,IFIT2,ENSG00000119922,3433,10,89283694.0,89309271.0,+
2,CXCL10,ENSG00000169245,3627,4,76021118.0,76023497.0,-
3,RSAD2,ENSG00000134321,91543,2,6865557.0,6898239.0,+
4,CXCL11,ENSG00000169248,6373,4,76033682.0,76041415.0,-
5,TNFSF10,ENSG00000121858,8743,3,172505508.0,172523475.0,-
...,...,...,...,...,...,...,...
23416,112268317,ENSG00000307470,112268317,NaN,NaN,NaN,NaN
23417,107987399,NaN,107987399,NaN,NaN,NaN,NaN
23418,105379561,NaN,105379561,NaN,NaN,NaN,NaN
23419,112267916,NaN,112267916,NaN,NaN,NaN,NaN


In [49]:
genes_trem2 = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Genes/trem2_genes.csv')
genes_trem2 = genes_trem2[['HGNC Symbol', 'Ensembl ID', 'Entrez ID', 'Chromosome', 'Start', 'End', 'Strand']]
genes_trem2.columns = ['hgnc', 'ensembl_gene_id', 'entrez', 'chromosome', 'start_position', 'end_position', 'strand']
genes_trem2

,hgnc,ensembl_gene_id,entrez,chromosome,start_position,end_position,strand
0,AKT3,ENSG00000275199,10000,HSCHR1_3_CTG32_1,500341,867542,-
1,AKT3,ENSG00000117020,10000,1,243488233,243851079,-
2,SNORA36C,ENSG00000207016,100124535,2,69520043,69520174,-
3,SNORA70B,ENSG00000206937,100124537,2,61417244,61417378,-
4,MIR216B,ENSG00000211520,100126319,2,56000714,56000795,-
...,...,...,...,...,...,...,...
30363,RBX1,ENSG00000100387,9978,22,40951347,40973309,+
30364,DOP1B,ENSG00000142197,9980,21,36156782,36294274,+
30365,KCNE2,ENSG00000159197,9992,21,34180729,34371381,+
30366,DGCR2,ENSG00000070413,9993,22,19036282,19122454,-


In [44]:
genes_coculture = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Genes/coculture_genes.csv')
genes_coculture['strand'] = genes_coculture['strand'].replace({1: '+', -1: '-'})
genes_coculture

,hgnc,ensembl_gene_id,entrez,chromosome,start_position,end_position,strand
0,ADA,ENSG00000196839,100,20,44584896,44652252,-
1,CDH2,ENSG00000170558,1000,18,27932879,28177946,-
2,RNA5-8SN5,ENSG00000274917,100008587,GL000220.1,112025,112177,+
3,SRA1,ENSG00000213523,10011,5,140537340,140557677,-
4,RANBP3-DT,ENSG00000266983,100128568,19,5978176,6020363,+
...,...,...,...,...,...,...,...
1268,KBTBD11,ENSG00000273645,9920,HSCHR8_8_CTG1,157254,190312,+
1269,KBTBD11,ENSG00000176595,9920,8,1973677,2006936,+
1270,PAN2,ENSG00000135473,9924,12,56316223,56334053,-
1271,KIF14,ENSG00000118193,9928,1,200551497,200620751,-


In [46]:
genes_xmg_7days = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Genes/gene_annotations_iMG_vs_xMG7days.csv')
genes_xmg_7days['strand'] = genes_xmg_7days['strand'].replace({1: '+', -1: '-'})
genes_xmg_7days = genes_xmg_7days[['hgnc', 'ensembl_gene_id', 'entrez', 'chromosome', 'start_position', 'end_position', 'strand']]
genes_xmg_7days

,hgnc,ensembl_gene_id,entrez,chromosome,start_position,end_position,strand
0,A1BG,ENSG00000121410,1,19,58345178,58353492,-
1,ADA,ENSG00000196839,100,20,44584896,44652252,-
2,AKT3,ENSG00000275199,10000,HSCHR1_3_CTG32_1,500341,867542,-
3,AKT3,ENSG00000117020,10000,1,243488233,243851079,-
4,RNA5-8SN5,ENSG00000274917,100008587,GL000220.1,112025,112177,+
...,...,...,...,...,...,...,...
4234,THOC1,ENSG00000079134,9984,18,214520,268050,-
4235,HNRNPDL,ENSG00000152795,9987,4,82422565,82430462,-
4236,CASP8AP2,ENSG00000288475,9994,HG2121_PATCH,25225,72124,+
4237,CASP8AP2,ENSG00000118412,9994,6,89829894,89874436,+


combine all rows based on entrez

In [50]:
# combine all the dfs based on unique entrez
combined_genes = pd.concat([genes_inf, genes_trem2, genes_coculture, genes_xmg_7days], ignore_index=True)
combined_genes = combined_genes.drop_duplicates(subset=['entrez'])
combined_genes

,hgnc,ensembl_gene_id,entrez,chromosome,start_position,end_position,strand
0,IFIT2,ENSG00000119922,3433,10,89283694.0,89309271.0,+
1,CXCL10,ENSG00000169245,3627,4,76021118.0,76023497.0,-
2,RSAD2,ENSG00000134321,91543,2,6865557.0,6898239.0,+
3,CXCL11,ENSG00000169248,6373,4,76033682.0,76041415.0,-
4,TNFSF10,ENSG00000121858,8743,3,172505508.0,172523475.0,-
...,...,...,...,...,...,...,...
53738,PAGE4,ENSG00000101951,9506,X,49793653.0,49834264.0,+
53741,GAL3ST1,ENSG00000128242,9514,22,30554635.0,30574665.0,-
53747,CD40LG,ENSG00000102245,959,X,136648158.0,136660390.0,+
53762,FRMPD4,ENSG00000169933,9758,X,11822423.0,12724523.0,+


In [51]:
combined_genes.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/combined_genes.csv', index=False)

## Differential Expression

DE: Combine all differential expression data tables, add in two columns for cell and condition

In [106]:
csv_dir = '/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/DE'
csv_files = glob.glob(os.path.join(csv_dir, '*.csv'))
for file in csv_files:
    df = pd.read_csv(file)
    print(f"Columns in {os.path.basename(file)}:")
    print(df.columns.tolist())
    print('-' * 40)

Columns in differential_expression_results_iMG_vs_oMG.csv:
['ensembl_gene_id', 'geneID', 'gene_symbol', 'gene_name', 'log2FoldChange', 'pvalue', 'padj', 'baseMean']
----------------------------------------
Columns in TREM2_KO_deseq.csv:
['geneID', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj', 'gene_symbol', 'description']
----------------------------------------
Columns in TREM2_R47H_deseq.csv:
['geneID', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj', 'gene_symbol', 'description']
----------------------------------------
Columns in WTC11_ifnb_DE.csv:
['Unnamed: 0', 'entrez', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']
----------------------------------------
Columns in differential_expression_results_iMG_vs_xMG-7days.csv:
['ensembl_gene_id', 'geneID', 'gene_symbol', 'gene_name', 'log2FoldChange', 'pvalue', 'padj', 'baseMean']
----------------------------------------
Columns in H1_ifnb_DE.csv:
['Unnamed: 0', 'entrez', 'baseMean', 'log2FoldChange'

In [10]:
wtc11_de = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/DE/WTC11_ifnb_DE.csv', index_col=0)
wtc11_de.insert(wtc11_de.columns.get_loc('entrez')+1, 'condition', 'IFN')
wtc11_de.insert(wtc11_de.columns.get_loc('entrez')+2, 'cell_type', 'iPSC')
wtc11_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
1,3433,IFN,iPSC,51363.949143,7.076655,2.431514e-161,3.661131e-157
2,3627,IFN,iPSC,31139.923458,7.208627,7.782382e-159,5.858966e-155
3,91543,IFN,iPSC,17487.285409,7.078944,1.216976e-151,6.108001e-148
4,6373,IFN,iPSC,8241.154933,8.971610,2.625696e-140,9.883776e-137
5,8743,IFN,iPSC,5885.554918,8.436834,4.660559e-135,1.403481e-131
...,...,...,...,...,...,...,...
23416,112268317,IFN,iPSC,8.925314,0.790385,6.401772e-01,NaN
23417,107987399,IFN,iPSC,11.602322,0.674284,6.483830e-01,NaN
23418,105379561,IFN,iPSC,3.020266,-1.957856,5.170214e-01,NaN
23419,112267916,IFN,iPSC,9.796965,-2.741803,1.495176e-01,NaN


In [11]:
h1_de = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/DE/H1_ifnb_DE.csv', index_col=0)
h1_de.insert(h1_de.columns.get_loc('entrez')+1, 'condition', 'IFN')
h1_de.insert(h1_de.columns.get_loc('entrez')+2, 'cell_type', 'ESC')
h1_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
1,91543,IFN,ESC,13475.331282,10.264425,1.447029e-268,2.130895e-264
2,8638,IFN,ESC,5899.382711,8.398069,1.567318e-220,1.154016e-216
3,4599,IFN,ESC,16428.403759,6.458007,3.290398e-207,1.615147e-203
4,3434,IFN,ESC,48908.668496,8.256053,4.550447e-192,1.675247e-188
5,3437,IFN,ESC,22345.434892,5.937422,1.197989e-182,3.528316e-179
...,...,...,...,...,...,...,...
23416,107987401,IFN,ESC,7.769690,-0.399492,7.454384e-01,NaN
23417,4572,IFN,ESC,3.285157,-0.687154,7.282761e-01,NaN
23418,4564,IFN,ESC,1.225285,-0.371544,8.955932e-01,NaN
23419,4575,IFN,ESC,0.000000,NaN,NaN,NaN


In [12]:
trem2ko_de = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/DE/TREM2_KO_deseq.csv')
trem2ko_de = trem2ko_de[['geneID', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']]
trem2ko_de.columns = ['entrez', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']
trem2ko_de.insert(trem2ko_de.columns.get_loc('entrez')+1, 'condition', 'TREM2KO')
trem2ko_de.insert(trem2ko_de.columns.get_loc('entrez')+2, 'cell_type', 'ESC')
trem2ko_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
0,23240,TREM2KO,ESC,390.675732,-0.351666,5.952899e-08,0.000990
1,998,TREM2KO,ESC,5228.274617,-0.162124,2.515882e-07,0.002092
2,2167,TREM2KO,ESC,20.332458,3.446080,4.977782e-07,0.002759
3,9262,TREM2KO,ESC,1589.645549,-0.392097,8.858416e-07,0.003682
4,8895,TREM2KO,ESC,1093.591798,-0.243021,1.999798e-06,0.006651
...,...,...,...,...,...,...,...
32815,102723847,TREM2KO,ESC,0.523504,0.637379,7.002371e-01,NaN
32816,105379417,TREM2KO,ESC,0.987051,3.401924,5.820441e-02,NaN
32817,105379566,TREM2KO,ESC,0.195688,0.430377,8.696057e-01,NaN
32818,107987401,TREM2KO,ESC,0.660971,-0.657456,6.176979e-01,NaN


In [13]:
trem2r47h_de = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/DE/TREM2_R47H_deseq.csv')
trem2r47h_de = trem2r47h_de[['geneID', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']]
trem2r47h_de.columns = ['entrez', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']
trem2r47h_de.insert(trem2r47h_de.columns.get_loc('entrez')+1, 'condition', 'TREM2R47H')
trem2r47h_de.insert(trem2r47h_de.columns.get_loc('entrez')+2, 'cell_type', 'ESC')
trem2r47h_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
0,29948,TREM2R47H,ESC,18.076160,1.361652,9.298600e-08,0.001378
1,64786,TREM2R47H,ESC,597.139516,-0.362428,1.413515e-06,0.010471
2,84002,TREM2R47H,ESC,2668.769621,-0.362971,2.633130e-06,0.013003
3,6461,TREM2R47H,ESC,175.548315,0.510790,5.694400e-06,0.016873
4,284129,TREM2R47H,ESC,217.989695,0.588959,5.190929e-06,0.016873
...,...,...,...,...,...,...,...
32815,102723847,TREM2R47H,ESC,0.132067,-0.702108,8.199424e-01,NaN
32816,105379417,TREM2R47H,ESC,0.565925,2.677743,3.765988e-01,NaN
32817,105379566,TREM2R47H,ESC,0.071876,0.534472,8.624372e-01,NaN
32818,107987401,TREM2R47H,ESC,0.507951,1.262275,3.873289e-01,NaN


In [14]:
xmg_de = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/DE/differential_expression_results_iMG_vs_xMG-7days.csv')
xmg_de = xmg_de[['geneID', 'baseMean','log2FoldChange', 'pvalue', 'padj']] 
xmg_de.columns = ['entrez', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']
xmg_de.insert(xmg_de.columns.get_loc('entrez')+1, 'condition', 'xenot7d')
xmg_de.insert(xmg_de.columns.get_loc('entrez')+2, 'cell_type', 'iPSC')
xmg_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
0,729737,xenot7d,iPSC,572.381215,-2.117348,5.900259e-04,2.629321e-03
1,100132287,xenot7d,iPSC,204.892806,-1.797802,6.878271e-07,6.552458e-06
2,100133331,xenot7d,iPSC,181.949972,-1.690312,8.825274e-06,6.415882e-05
3,79854,xenot7d,iPSC,11.200773,-2.080607,6.066192e-03,1.966917e-02
4,643837,xenot7d,iPSC,119.555942,-1.054365,1.436130e-03,5.658187e-03
...,...,...,...,...,...,...,...
4203,4541,xenot7d,iPSC,9611.938348,-1.625844,1.547930e-08,2.126153e-07
4204,4556,xenot7d,iPSC,465.777487,-1.544319,1.061511e-06,9.720227e-06
4205,4519,xenot7d,iPSC,28681.291675,-1.450410,4.936545e-08,6.156218e-07
4206,4576,xenot7d,iPSC,84.263876,1.042201,2.857335e-04,1.388658e-03


In [19]:
omg_de = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/DE/differential_expression_results.csv')
omg_de = omg_de[['geneID', 'baseMean','log2FoldChange', 'pvalue', 'padj']]
omg_de.columns = ['entrez', 'baseMean', 'log2FoldChange', 'pvalue', 'padj']
omg_de.insert(omg_de.columns.get_loc('entrez')+1, 'condition', 'coculture')
omg_de.insert(omg_de.columns.get_loc('entrez')+2, 'cell_type', 'iPSC')
omg_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
0,57801,coculture,iPSC,3.226657,3.876730,9.604124e-04,1.184946e-02
1,375790,coculture,iPSC,64.409736,1.640842,2.947100e-04,4.969108e-03
2,112267874,coculture,iPSC,3.098629,3.224100,5.706338e-03,4.244529e-02
3,388591,coculture,iPSC,70.581749,1.695869,4.529711e-04,6.805862e-03
4,387509,coculture,iPSC,13.714961,1.985986,2.471307e-03,2.388710e-02
...,...,...,...,...,...,...,...
1282,4570,coculture,iPSC,3017.339959,-1.042412,3.818550e-06,1.901385e-04
1283,4555,coculture,iPSC,74.789086,1.586012,4.269662e-11,1.484118e-08
1284,4509,coculture,iPSC,5001.407250,1.512440,6.821825e-14,4.403732e-11
1285,4508,coculture,iPSC,35394.848958,1.197526,2.689226e-12,1.429640e-09


In [20]:
combiend_de = pd.concat([wtc11_de, h1_de, trem2ko_de, trem2r47h_de, xmg_de, omg_de], ignore_index=True)
combiend_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
0,3433,IFN,iPSC,51363.949143,7.076655,2.431514e-161,3.661131e-157
1,3627,IFN,iPSC,31139.923458,7.208627,7.782382e-159,5.858966e-155
2,91543,IFN,iPSC,17487.285409,7.078944,1.216976e-151,6.108001e-148
3,6373,IFN,iPSC,8241.154933,8.971610,2.625696e-140,9.883776e-137
4,8743,IFN,iPSC,5885.554918,8.436834,4.660559e-135,1.403481e-131
...,...,...,...,...,...,...,...
117970,4570,coculture,iPSC,3017.339959,-1.042412,3.818550e-06,1.901385e-04
117971,4555,coculture,iPSC,74.789086,1.586012,4.269662e-11,1.484118e-08
117972,4509,coculture,iPSC,5001.407250,1.512440,6.821825e-14,4.403732e-11
117973,4508,coculture,iPSC,35394.848958,1.197526,2.689226e-12,1.429640e-09


In [25]:
unique_de = combiend_de.drop_duplicates(subset=['entrez', 'condition', 'cell_type'])
unique_de

,entrez,condition,cell_type,baseMean,log2FoldChange,pvalue,padj
0,3433,IFN,iPSC,51363.949143,7.076655,2.431514e-161,3.661131e-157
1,3627,IFN,iPSC,31139.923458,7.208627,7.782382e-159,5.858966e-155
2,91543,IFN,iPSC,17487.285409,7.078944,1.216976e-151,6.108001e-148
3,6373,IFN,iPSC,8241.154933,8.971610,2.625696e-140,9.883776e-137
4,8743,IFN,iPSC,5885.554918,8.436834,4.660559e-135,1.403481e-131
...,...,...,...,...,...,...,...
117970,4570,coculture,iPSC,3017.339959,-1.042412,3.818550e-06,1.901385e-04
117971,4555,coculture,iPSC,74.789086,1.586012,4.269662e-11,1.484118e-08
117972,4509,coculture,iPSC,5001.407250,1.512440,6.821825e-14,4.403732e-11
117973,4508,coculture,iPSC,35394.848958,1.197526,2.689226e-12,1.429640e-09


In [27]:
combiend_de.to_csv('combined_de.csv', index=False)

## Biological Pathways

In [83]:
pathways = pd.read_csv('/Users/nathan/Desktop/Lei\'s Lab/AD_Microglia_DE_Analysis/AD_Microglia_DEGs/DEA/biological_pathways.csv', index_col=0)
pathways['pathways'] = pathways['pathways'].str.replace('_',' ')
pathways

,pathways
1,SA B CELL RECEPTOR COMPLEXES
2,SA CASPASE CASCADE
3,SA FAS SIGNALING
4,SA G1 AND S PHASES
5,SA G2 AND M PHASES
...,...
3913,WP WNT SIGNALING AND PLURIPOTENCY
3914,WP WNT SIGNALING IN KIDNEY DISEASE
3915,WP WNT SIGNALING WP363
3916,WP WNT SIGNALING WP428


In [84]:
pathways.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/pathways.csv', index=False)

## Pathways - Genes

In [74]:
gsea_dir = '/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/GSEA'
csv_files = glob.glob(os.path.join(gsea_dir, '*.csv'))
for file in csv_files:
    df = pd.read_csv(file)
    print(f"Columns in {os.path.basename(file)}:")
    print(df.columns.tolist())
    print('-' * 40) 

Columns in wtc11_ifnb_gsea.csv:
['Unnamed: 0', 'pathway', 'pval', 'padj', 'log2err', 'ES', 'NES', 'size', 'gene']
----------------------------------------
Columns in TREM2_R47H_gsea_genes_c2.csv:
['pathway', 'NES', 'padj', 'gene_id']
----------------------------------------
Columns in pathway_genes_c2_canonical_iMG_vs_xMG7days.csv:
['ensembl_gene_id', 'gs_name', 'entrez_gene', 'gene_symbol', 'pathway', 'clean_pathway_name', 'nes', 'padj']
----------------------------------------
Columns in TREM2_KO_gsea_genes_c2.csv:
['pathway', 'NES', 'padj', 'gene_id']
----------------------------------------
Columns in h1_ifnb_gsea.csv:
['Unnamed: 0', 'pathway', 'pval', 'padj', 'log2err', 'ES', 'NES', 'size', 'gene']
----------------------------------------
Columns in pathway_genes_c2_canonical_iMG_vs_oMG1.csv:
['ensembl_gene_id', 'gs_name', 'entrez_gene', 'gene_symbol', 'pathway', 'clean_pathway_name', 'nes', 'padj']
----------------------------------------


In [94]:
wtc11_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/GSEA/wtc11_ifnb_gsea.csv', index_col=0)
wtc11_pathways = wtc11_pathways[['gene', 'pathway']]
wtc11_pathways['gene'] = wtc11_pathways['gene'].str.replace("\"", '')
wtc11_pathways['pathway'] = wtc11_pathways['pathway'].str.replace("_", ' ')
wtc11_pathways.columns = ['entrez', 'pathway']
wtc11_pathways

,entrez,pathway
1,3669,REACTOME INTERFERON ALPHA BETA SIGNALING
2,91543,REACTOME INTERFERON ALPHA BETA SIGNALING
3,3433,REACTOME INTERFERON ALPHA BETA SIGNALING
4,3429,REACTOME INTERFERON ALPHA BETA SIGNALING
5,3434,REACTOME INTERFERON ALPHA BETA SIGNALING
...,...,...
38082,6171,KEGG MEDICUS REFERENCE TRANSLATION INITIATION
38083,6208,KEGG MEDICUS REFERENCE TRANSLATION INITIATION
38084,6167,KEGG MEDICUS REFERENCE TRANSLATION INITIATION
38085,6187,KEGG MEDICUS REFERENCE TRANSLATION INITIATION


In [95]:
h1_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/GSEA/h1_ifnb_gsea.csv')
h1_pathways = h1_pathways[['gene', 'pathway']]
h1_pathways['gene'] = h1_pathways['gene'].str.replace("\"", '')
h1_pathways['pathway'] = h1_pathways['pathway'].str.replace("_", ' ')
h1_pathways.columns = ['entrez', 'pathway']
h1_pathways

,entrez,pathway
0,91543,REACTOME INTERFERON ALPHA BETA SIGNALING
1,3669,REACTOME INTERFERON ALPHA BETA SIGNALING
2,8638,REACTOME INTERFERON ALPHA BETA SIGNALING
3,3434,REACTOME INTERFERON ALPHA BETA SIGNALING
4,4939,REACTOME INTERFERON ALPHA BETA SIGNALING
...,...,...
35540,3990,REACTOME PLASMA LIPOPROTEIN REMODELING
35541,4547,REACTOME PLASMA LIPOPROTEIN REMODELING
35542,3931,REACTOME PLASMA LIPOPROTEIN REMODELING
35543,348,REACTOME PLASMA LIPOPROTEIN REMODELING


In [96]:
trem2ko_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/GSEA/TREM2_KO_gsea_genes_c2.csv')
trem2ko_pathways = trem2ko_pathways[['gene_id', 'pathway']]
trem2ko_pathways['pathway'] = trem2ko_pathways['pathway'].str.replace("_", ' ')
trem2ko_pathways.columns = ['entrez', 'pathway']
trem2ko_pathways

,entrez,pathway
0,53632,REACTOME ENERGY DEPENDENT REGULATION OF MTOR B...
1,5563,REACTOME ENERGY DEPENDENT REGULATION OF MTOR B...
2,6794,REACTOME ENERGY DEPENDENT REGULATION OF MTOR B...
3,389541,REACTOME ENERGY DEPENDENT REGULATION OF MTOR B...
4,64223,REACTOME ENERGY DEPENDENT REGULATION OF MTOR B...
...,...,...
45698,1305,WP VITAMIN D RECEPTOR PATHWAY
45699,57167,WP VITAMIN D RECEPTOR PATHWAY
45700,820,WP VITAMIN D RECEPTOR PATHWAY
45701,200407,WP VITAMIN D RECEPTOR PATHWAY


In [97]:
trem2r47h_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/GSEA/TREM2_R47H_gsea_genes_c2.csv')
trem2r47h_pathways = trem2r47h_pathways[['gene_id', 'pathway']]
trem2r47h_pathways['pathway'] = trem2r47h_pathways['pathway'].str.replace("_", ' ')
trem2r47h_pathways.columns = ['entrez', 'pathway']
trem2r47h_pathways

,entrez,pathway
0,2167,REACTOME TRIGLYCERIDE METABOLISM
1,2172,REACTOME TRIGLYCERIDE METABOLISM
2,346606,REACTOME TRIGLYCERIDE METABOLISM
3,392636,REACTOME TRIGLYCERIDE METABOLISM
4,5346,REACTOME TRIGLYCERIDE METABOLISM
...,...,...
43152,10581,REACTOME INTERFERON ALPHA BETA SIGNALING
43153,3664,REACTOME INTERFERON ALPHA BETA SIGNALING
43154,6772,REACTOME INTERFERON ALPHA BETA SIGNALING
43155,5610,REACTOME INTERFERON ALPHA BETA SIGNALING


In [98]:
xenot7d_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/GSEA/pathway_genes_c2_canonical_iMG_vs_xMG7days.csv')
xenot7d_pathways = xenot7d_pathways[['entrez_gene', 'clean_pathway_name']]
xenot7d_pathways.columns = ['entrez', 'pathway']
xenot7d_pathways


,entrez,pathway
0,1636,SA MMP CYTOKINE CONNECTION
1,960,SA MMP CYTOKINE CONNECTION
2,2214,SA MMP CYTOKINE CONNECTION
3,944,SA MMP CYTOKINE CONNECTION
4,2318,SIG REGULATION OF THE ACTIN CYTOSKELETON BY RH...
...,...,...
105,23533,SIG PIP3 SIGNALING IN B LYMPHOCYTES
106,23368,SIG PIP3 SIGNALING IN B LYMPHOCYTES
107,57580,SIG PIP3 SIGNALING IN B LYMPHOCYTES
108,6295,SIG PIP3 SIGNALING IN B LYMPHOCYTES


In [99]:
coculture_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/GSEA/pathway_genes_c2_canonical_iMG_vs_oMG1.csv')
coculture_pathways = coculture_pathways[['entrez_gene', 'clean_pathway_name']]
coculture_pathways.columns = ['entrez', 'pathway']
coculture_pathways

,entrez,pathway
0,85366,SIG REGULATION OF THE ACTIN CYTOSKELETON BY RH...
1,5063,SIG REGULATION OF THE ACTIN CYTOSKELETON BY RH...
2,57144,SIG REGULATION OF THE ACTIN CYTOSKELETON BY RH...
3,5600,SIG CD40PATHWAYMAP
4,5603,SIG CD40PATHWAYMAP
...,...,...
115,4067,SA B CELL RECEPTOR COMPLEXES
116,5604,SA B CELL RECEPTOR COMPLEXES
117,6197,SA B CELL RECEPTOR COMPLEXES
118,7409,SA B CELL RECEPTOR COMPLEXES


In [101]:
combined_gene_pathways = pd.concat([wtc11_pathways, h1_pathways, trem2ko_pathways, trem2r47h_pathways, xenot7d_pathways, coculture_pathways], ignore_index=True)
combined_gene_pathways = combined_gene_pathways.drop_duplicates(subset=['entrez', 'pathway'])
print(len(combined_gene_pathways))
combined_gene_pathways = combined_gene_pathways.dropna()
print(len(combined_gene_pathways))
combined_gene_pathways

114211
114211


,entrez,pathway
0,3669,REACTOME INTERFERON ALPHA BETA SIGNALING
1,91543,REACTOME INTERFERON ALPHA BETA SIGNALING
2,3433,REACTOME INTERFERON ALPHA BETA SIGNALING
3,3429,REACTOME INTERFERON ALPHA BETA SIGNALING
4,3434,REACTOME INTERFERON ALPHA BETA SIGNALING
...,...,...
162655,5336,SIG PIP3 SIGNALING IN B LYMPHOCYTES
162663,7409,SIG PIP3 SIGNALING IN B LYMPHOCYTES
162692,207,SA PTEN PATHWAY
162697,2885,SA PTEN PATHWAY


In [102]:
combined_gene_pathways.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/combined_gene_pathways.csv', index=False)

## CREs

In [ ]:
directory = "./Data/CREs"
for file in os.listdir(directory):
    if file.endswith(".txt"):
        filepath = os.path.join(directory, file)
        df = pd.read_csv(filepath, sep='\t')
        df = df.iloc[:, 0:6]
        df.columns = ["unknown", "RPM_condition", "RPM_control", "chr", "start", "end"]
        df['CRElog2FC'] = np.log2((df["RPM_condition"] + 1) / (df["RPM_control"] + 1))
        df.to_csv(filepath, sep="\t", index=False)

In [7]:
for file in os.listdir(directory):
    if file.endswith(".txt"):
        filepath = os.path.join(directory, file)
        df = pd.read_csv(filepath, sep='\t')
        df = df.iloc[:, 3:7]
        df.to_csv(filepath, sep="\t", index=False)

Read mapped cres in and perform metadata changes

In [64]:
directory = "./Data/CREs"
outdir1 = "./Data/Filtered_cres"
outdir2 = "./Data/Mapped_cres"
for file in os.listdir(directory):
    if file.endswith(".tsv"):
        filepath = os.path.join(directory, file)
        df = pd.read_csv(filepath, sep='\t')
        df.columns = ["chr", "start", "end", "cre_log2foldchange", "merged_chr", "merged_start", "merged_end", "tss_chr", "tss_start", "tss_end", "strand", "hgnc", "entrez", "na1", "na2", "distance_to_tss"]
        if file == "h1_mapped_cres.tsv":
            df.insert(0, 'condition', 'IFN')
            df.insert(1, 'cell_type', 'ESC')
        elif file == "wtc11_mapped_cres.tsv":
            df.insert(0, 'condition', 'IFN')
            df.insert(1, 'cell_type', 'iPSC')
        elif file == "trem2ko_mapped_cres.tsv":
            df.insert(0, 'condition', 'TREM2KO')
            df.insert(1, 'cell_type', 'ESC')
        elif file == "trem2r47h_mapped_cres.tsv":
            df.insert(0, 'condition', 'TREM2R47H')
            df.insert(1, 'cell_type', 'ESC')
        elif file == "xenot7d_mapped_cres.tsv":
            df.insert(0, 'condition', 'xenot7d')
            df.insert(1, 'cell_type', 'iPSC')
        elif file == "coculture_mapped_cres.tsv":
            df.insert(0, 'condition', 'coculture')
            df.insert(1, 'cell_type', 'iPSC')
        cres = df[['condition', 'cell_type', 'chr', 'start', 'end', 'cre_log2foldchange', "merged_chr", "merged_start", "merged_end"]]
        cres_mapped = df[['condition', 'cell_type', 'chr', 'start', 'end', "entrez", "distance_to_tss"]]
        outpath1 = os.path.join(outdir1, file)
        outpath2 = os.path.join(outdir2, file)
        cres.to_csv(outpath1, sep='\t', index=False)
        cres_mapped.to_csv(outpath2, sep='\t', index=False)

Read in filtered cres, combine them into one table and drop duplicates

In [65]:
h1_cres = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Filtered_cres/h1_mapped_cres.tsv", sep="\t")
wtc11_cres = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Filtered_cres/wtc11_mapped_cres.tsv", sep="\t")
trem2ko_cres = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Filtered_cres/trem2ko_mapped_cres.tsv", sep="\t")
trem2r47h_cres = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Filtered_cres/trem2r47h_mapped_cres.tsv", sep="\t")
xenot7d_cres = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Filtered_cres/xenot7d_mapped_cres.tsv", sep="\t")
coculture_cres = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Filtered_cres/coculture_mapped_cres.tsv", sep="\t")

In [66]:
cres = pd.concat([h1_cres, wtc11_cres, trem2ko_cres, trem2r47h_cres, xenot7d_cres, coculture_cres], ignore_index=True)
cres

,condition,cell_type,chr,start,end,cre_log2foldchange,merged_chr,merged_start,merged_end
0,IFN,ESC,chr10,89295608,89296790,0.434959,chr10,89295506,89297078
1,IFN,ESC,chr10,89301577,89302330,-0.104734,chr10,89301099,89303218
2,IFN,ESC,chr10,89309356,89310733,2.769050,chr10,89309356,89310733
3,IFN,ESC,chr2,6865106,6866522,0.597089,chr2,6863605,6867024
4,IFN,ESC,chr2,6877401,6878097,2.152570,chr2,6875992,6878409
...,...,...,...,...,...,...,...,...,...
361158,coculture,iPSC,chrX,11996722,11997282,-0.299677,chrX,11996623,11997551
361159,coculture,iPSC,chrX,12301160,12301819,-1.798817,chrX,12301069,12301966
361160,coculture,iPSC,chrX,12498107,12498700,-1.062652,chrX,12497991,12498808
361161,coculture,iPSC,chrX,12506691,12507175,-0.404536,chrX,12506606,12507175


In [67]:
unique_cres = cres.drop_duplicates(subset=['condition', 'cell_type', 'chr', 'start', 'end'])
unique_cres['chr'] = unique_cres['chr'].str.replace('chr', '')
unique_cres['merged_chr'] = unique_cres['merged_chr'].str.replace('chr', '')
unique_cres

/var/folders/y_/tfywlws16k71l2k62r0j683m0000gn/T/ipykernel_95578/501924833.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_cres['chr'] = unique_cres['chr'].str.replace('chr', '')
/var/folders/y_/tfywlws16k71l2k62r0j683m0000gn/T/ipykernel_95578/501924833.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_cres['merged_chr'] = unique_cres['merged_chr'].str.replace('chr', '')


,condition,cell_type,chr,start,end,cre_log2foldchange,merged_chr,merged_start,merged_end
0,IFN,ESC,10,89295608,89296790,0.434959,10,89295506,89297078
1,IFN,ESC,10,89301577,89302330,-0.104734,10,89301099,89303218
2,IFN,ESC,10,89309356,89310733,2.769050,10,89309356,89310733
3,IFN,ESC,2,6865106,6866522,0.597089,2,6863605,6867024
4,IFN,ESC,2,6877401,6878097,2.152570,2,6875992,6878409
...,...,...,...,...,...,...,...,...,...
361158,coculture,iPSC,X,11996722,11997282,-0.299677,X,11996623,11997551
361159,coculture,iPSC,X,12301160,12301819,-1.798817,X,12301069,12301966
361160,coculture,iPSC,X,12498107,12498700,-1.062652,X,12497991,12498808
361161,coculture,iPSC,X,12506691,12507175,-0.404536,X,12506606,12507175


In [68]:
unique_cres.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/unique_cres.csv', index=False)

now for cres linked to genes

In [69]:
h1_cres_genes = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Mapped_cres/h1_mapped_cres.tsv", sep="\t")
wtc11_cres_genes = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Mapped_cres/wtc11_mapped_cres.tsv", sep="\t")
trem2ko_cres_genes = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Mapped_cres/trem2ko_mapped_cres.tsv", sep="\t")
trem2r47h_cres_genes = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Mapped_cres/trem2r47h_mapped_cres.tsv", sep="\t")
xenot7d_cres_genes = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Mapped_cres/xenot7d_mapped_cres.tsv", sep="\t")
coculture_cres_genes = pd.read_csv("/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Data/Mapped_cres/coculture_mapped_cres.tsv", sep="\t")

In [70]:
unique_cres_genes = pd.concat([h1_cres_genes, wtc11_cres_genes, trem2ko_cres_genes, trem2r47h_cres_genes, xenot7d_cres_genes, coculture_cres_genes], ignore_index=True)
print(len(unique_cres_genes))
unique_cres_genes = unique_cres_genes.drop_duplicates(subset=['condition', 'cell_type','chr', 'start', 'end', 'entrez'])
print(len(unique_cres_genes))
unique_cres_genes['chr'] = unique_cres_genes['chr'].str.replace('chr', '')
unique_cres_genes

361163
360526


,condition,cell_type,chr,start,end,entrez,distance_to_tss
0,IFN,ESC,10,89295608,89296790,3433,3087
1,IFN,ESC,10,89301577,89302330,3433,0
2,IFN,ESC,10,89309356,89310733,3433,3395
3,IFN,ESC,2,6865106,6866522,91543,0
4,IFN,ESC,2,6877401,6878097,91543,0
...,...,...,...,...,...,...,...
361158,coculture,iPSC,X,11996722,11997282,9758,118862
361159,coculture,iPSC,X,12301160,12301819,9758,39726
361160,coculture,iPSC,X,12498107,12498700,9758,3143
361161,coculture,iPSC,X,12506691,12507175,9758,11727


In [72]:
duplicates = unique_cres_genes.groupby(['start', 'end']) \
                              .filter(lambda x: len(x) > 1)

In [73]:
duplicates

,condition,cell_type,chr,start,end,entrez,distance_to_tss
3,IFN,ESC,2,6865106,6866522,91543,0
14,IFN,ESC,3,57227304,57228821,8820,0
15,IFN,ESC,2,6865106,6866522,129607,0
39,IFN,ESC,2,162318011,162319366,64135,0
42,IFN,ESC,6,32853316,32854532,5698,0
...,...,...,...,...,...,...,...
361158,coculture,iPSC,X,11996722,11997282,9758,118862
361159,coculture,iPSC,X,12301160,12301819,9758,39726
361160,coculture,iPSC,X,12498107,12498700,9758,3143
361161,coculture,iPSC,X,12506691,12507175,9758,11727


In [71]:
unique_cres_genes.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/unique_cres_genes.csv', index=False) 

## Final Checks

In [67]:
final_genes = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/combined_genes.csv')
print(len(final_genes))
#final_genes = final_genes[final_genes['entrez'].isna()]
final_genes

31020


,hgnc,ensembl_gene_id,entrez,chromosome,start_position,end_position,strand
0,IFIT2,ENSG00000119922,3433,10,89283694.0,89309271.0,+
1,CXCL10,ENSG00000169245,3627,4,76021118.0,76023497.0,-
2,RSAD2,ENSG00000134321,91543,2,6865557.0,6898239.0,+
3,CXCL11,ENSG00000169248,6373,4,76033682.0,76041415.0,-
4,TNFSF10,ENSG00000121858,8743,3,172505508.0,172523475.0,-
...,...,...,...,...,...,...,...
31015,PAGE4,ENSG00000101951,9506,X,49793653.0,49834264.0,+
31016,GAL3ST1,ENSG00000128242,9514,22,30554635.0,30574665.0,-
31017,CD40LG,ENSG00000102245,959,X,136648158.0,136660390.0,+
31018,FRMPD4,ENSG00000169933,9758,X,11822423.0,12724523.0,+


In [34]:
final_cell_de = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/combined_de.csv')
print(len(final_cell_de))
final_cell_de = final_cell_de.drop_duplicates(subset=['entrez', 'condition', 'cell_type'])
cross1 = pd.merge(final_genes, final_cell_de, on='entrez', how='left')
cross1_na = cross1[cross1['pvalue'].isna()]
cross1_na   

117975


,hgnc,ensembl_gene_id,entrez,chromosome,start_position,end_position,strand,condition,cell_type,baseMean,log2FoldChange,pvalue,padj


In [65]:
final_cres = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/unique_cres.csv')
print(len(final_cres))
final_cres = final_cres.drop_duplicates(subset=['condition', 'cell_type','chr', 'start', 'end'])
print(len(final_cres))
#final_cres = final_cres[final_cres[['chr','start', 'end']].isna().any(axis=1)]
final_cres

340983
340983


,condition,cell_type,chr,start,end,cre_log2foldchange,merged_chr,merged_start,merged_end
0,IFN,ESC,10,89295608,89296790,0.434959,10,89295506,89297078
1,IFN,ESC,10,89301577,89302330,-0.104734,10,89301099,89303218
2,IFN,ESC,10,89309356,89310733,2.769050,10,89309356,89310733
3,IFN,ESC,2,6865106,6866522,0.597089,2,6863605,6867024
4,IFN,ESC,2,6877401,6878097,2.152570,2,6875992,6878409
...,...,...,...,...,...,...,...,...,...
340978,coculture,iPSC,X,11996722,11997282,-0.299677,X,11996623,11997551
340979,coculture,iPSC,X,12301160,12301819,-1.798817,X,12301069,12301966
340980,coculture,iPSC,X,12498107,12498700,-1.062652,X,12497991,12498808
340981,coculture,iPSC,X,12506691,12507175,-0.404536,X,12506606,12507175


In [36]:
final_merged = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/merged_cres.csv')
print(len(final_merged))
final_merged = final_merged.drop_duplicates(subset=['chr', 'start', 'end'])
print(len(final_merged))
final_merged

200049
200049


,chr,start,end
0,1,797406,798178
1,1,816037,818244
2,1,819296,820336
3,1,826435,828307
4,1,842686,843233
...,...,...,...
200044,Y,21421806,21422749
200045,Y,21440434,21441017
200046,Y,21481413,21482088
200047,Y,26408715,26409392


In [70]:
final_cre_gene = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/unique_cres_genes.csv')
print(len(final_cre_gene))
final_cre_gene = final_cre_gene[final_cre_gene[['chr','start', 'end']].notna().any(axis=1)]
print(len(final_cre_gene))
final_cre_gene = final_cre_gene.drop_duplicates(subset=['condition', 'cell_type','chr', 'start', 'end','entrez'])
print(len(final_cre_gene))
cross2 = pd.merge(final_cre_gene, final_cres, on=['condition', 'cell_type','chr', 'start', 'end'], how='inner')
cross2 = cross2[cross2['cre_log2foldchange'].isna()]
print(len(cross2))
cross3 = pd.merge(final_cre_gene, final_genes, on=['entrez'], how='left')
cross3 = cross3[cross3['strand'].isna()]
print(len(cross3))

360526
360526
360526
0
0


In [69]:
cross3

,condition,cell_type,chr,start,end,entrez,distance_to_tss,hgnc,ensembl_gene_id,chromosome,start_position,end_position,strand
23604,IFN,ESC,2,63124152,63124596,100169989,4611,NaN,ENSG00000293077,2,63117851.0,63119542.0,-
23614,IFN,ESC,2,7430624,7431506,100506274,620,NaN,ENSG00000229727,2,7293688.0,7450544.0,+
23650,IFN,ESC,1,60759216,60759812,101926964,65772,NaN,ENSG00000231252,1,60659631.0,60868041.0,-
23651,IFN,ESC,1,60866777,60867731,101926964,235,NaN,ENSG00000231252,1,60659631.0,60868041.0,-
23652,IFN,ESC,1,60874520,60875359,101926964,6512,NaN,ENSG00000231252,1,60659631.0,60868041.0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
360156,coculture,iPSC,21,9839168,9839451,441058,1158,NaN,ENSG00000293035,21,9781848.0,9839362.0,-
360346,coculture,iPSC,22,22558948,22559646,648691,0,NaN,ENSG00000220891,22,22559343.0,22566602.0,+
360409,coculture,iPSC,22,45996358,45998234,730668,8382,NaN,ENSG00000280424,22,46006616.0,46010777.0,+
360410,coculture,iPSC,22,46000867,46001379,730668,5237,NaN,ENSG00000280424,22,46006616.0,46010777.0,+


In [ ]:
final_merged_tfs = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/merged_cre_tf.csv')
print(len(final_merged_tfs))
final_merged_tfs = final_merged_tfs.drop_duplicates(subset=['chr', 'start', 'end', 'transcription_factor', 'cell_type', 'condition'])
print(len(final_merged_tfs))
final_merged_tfs = final_merged_tfs[final_merged_tfs[['chr','start', 'end','transcription_factor', 'cell_type', 'condition']].notna().any(axis=1)]
print(len(final_merged_tfs))


5686434
5608716
5608716


In [51]:
cross4 = pd.merge(final_merged_tfs, final_merged, on=['chr', 'start', 'end'], how='left')
cross4 = cross4[cross4['chr'].isna()]
cross4

,chr,start,end,transcription_factor,cell_type,condition


In [44]:
final_merged_tfs.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/merged_cre_tf.csv', index=False)

In [77]:
final_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/pathways.csv')
final_gene_pathways = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/combined_gene_pathways.csv')
print(len(final_gene_pathways))
final_gene_pathways = final_gene_pathways.drop_duplicates(subset=['entrez', 'pathway'])
print(len(final_gene_pathways))
final_gene_pathways = final_gene_pathways[final_gene_pathways[['entrez','pathway']].notna().any(axis=1)]
print(len(final_gene_pathways))
cross5 = pd.merge(final_gene_pathways, final_genes, on=['entrez'], how='left')
cols = ['hgnc', 'ensembl_gene_id', 'chromosome', 'start_position', 'end_position', 'strand']
cross5 = cross5[cross5[cols].isna().all(axis=1)]
print(len(cross5))

114211
83923
83923
138


In [80]:
excluded_entrez_ids = cross5['entrez'].dropna().unique()
excluded_entrez_ids
filtered_gene_pathways = final_gene_pathways[~final_gene_pathways['entrez'].isin(excluded_entrez_ids)]
filtered_gene_pathways

,entrez,pathway
0,3669,REACTOME INTERFERON ALPHA BETA SIGNALING
1,91543,REACTOME INTERFERON ALPHA BETA SIGNALING
2,3433,REACTOME INTERFERON ALPHA BETA SIGNALING
3,3429,REACTOME INTERFERON ALPHA BETA SIGNALING
4,3434,REACTOME INTERFERON ALPHA BETA SIGNALING
...,...,...
114198,6932,WNT SIGNALING
114200,2066,SIG PIP3 SIGNALING IN CARDIAC MYOCTES
114201,57144,SIG PIP3 SIGNALING IN CARDIAC MYOCTES
114206,5336,SIG PIP3 SIGNALING IN B LYMPHOCYTES


In [81]:
filtered_gene_pathways.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/filtered_gene_pathways.csv', index=False)

In [82]:
final_tfs = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/tfs.csv')
print(len(final_tfs))
final_tfs = final_tfs.drop_duplicates(subset=['transcription_factor'])
print(len(final_tfs))

525
519


In [3]:
m1 = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/merged_cre_tf.csv')
m1


,chr,start,end,transcription_factor,cell_type,condition
0,1,826435,828307,AHRARNT,ESC,IFN
1,1,1397971,1400475,AHRARNT,ESC,IFN
2,1,1614037,1616691,AHRARNT,ESC,IFN
3,1,1745525,1747386,AHRARNT,ESC,IFN
4,1,2226872,2228282,AHRARNT,ESC,IFN
...,...,...,...,...,...,...
5608711,Y,13751202,13752042,ZFP961,iPSC,coculture
5608712,Y,14523314,14525373,ZFP961,iPSC,coculture
5608713,Y,14840305,14840848,ZFP961,iPSC,coculture
5608714,Y,19075637,19076488,ZFP961,iPSC,coculture


In [4]:
m1 = m1[['chr', 'start', 'end', 'transcription_factor','condition','cell_type']]
m1

,chr,start,end,transcription_factor,condition,cell_type
0,1,826435,828307,AHRARNT,IFN,ESC
1,1,1397971,1400475,AHRARNT,IFN,ESC
2,1,1614037,1616691,AHRARNT,IFN,ESC
3,1,1745525,1747386,AHRARNT,IFN,ESC
4,1,2226872,2228282,AHRARNT,IFN,ESC
...,...,...,...,...,...,...
5608711,Y,13751202,13752042,ZFP961,coculture,iPSC
5608712,Y,14523314,14525373,ZFP961,coculture,iPSC
5608713,Y,14840305,14840848,ZFP961,coculture,iPSC
5608714,Y,19075637,19076488,ZFP961,coculture,iPSC


In [6]:
m1.to_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/merged_cre_tf.csv', index=False)

## Post Checks

In [2]:
u1 = pd.read_csv('/Users/nathan/Desktop/Bioinformatics/Projects/AD_Database/Tables/unique_cres_genes.csv')

In [3]:
u1_3087 = u1[u1['distance_to_tss'] == 3087]
u1_3087 

,condition,cell_type,chr,start,end,entrez,distance_to_tss
0,IFN,ESC,10,89295608,89296790,3433,3087
41773,IFN,iPSC,7,107907754,107908346,1738,3087
97596,TREM2KO,ESC,10,86714950,86715643,11155,3087
100223,TREM2KO,ESC,20,9397627,9398557,5332,3087
101325,TREM2KO,ESC,1,15434533,15435355,11330,3087
102516,TREM2KO,ESC,2,119641448,119642807,200373,3087
142021,TREM2R47H,ESC,19,49331583,49332084,951,3087
150772,TREM2R47H,ESC,10,86714950,86715643,11155,3087
153429,TREM2R47H,ESC,20,9397627,9398557,5332,3087
154525,TREM2R47H,ESC,1,15434533,15435355,11330,3087


In [5]:
u2 = u1[(u1['entrez'] == 3433) & (u1['condition'] == 'coculture')]
u2

,condition,cell_type,chr,start,end,entrez,distance_to_tss
261570,coculture,iPSC,10,89282690,89283817,3433,0
261571,coculture,iPSC,10,89295629,89296902,3433,3108
261572,coculture,iPSC,10,89301191,89302637,3433,0
261573,coculture,iPSC,10,89302738,89303106,3433,693
